In [1]:
import torch

## Starting with Tensors

In [2]:
t1 = torch.tensor(4.)
t1

tensor(4.)

In [3]:
t1.dtype

torch.float32

In [9]:
t2 = torch.tensor([1.,2.,3.,4.])

In [10]:
t2

tensor([1., 2., 3., 4.])

In [11]:
t3 = torch.tensor([[1,2],[3,4],[4,5]])

In [12]:
t1.shape

torch.Size([])

In [13]:
t2.shape

torch.Size([4])

In [14]:
t3.shape

torch.Size([3, 2])

## Tensor Operations and Gradients

In [15]:
x = torch.tensor(3.)
w = torch.tensor(4.,requires_grad = True)
b = torch.tensor(5.,requires_grad = True)

We can automatically calculate the gradient with respect to a variable once the 'requires_grad' option is specified

In [16]:
y = x*w + b

In [17]:
y

tensor(17., grad_fn=<AddBackward0>)

In [18]:
# Calcuate derivatives
y.backward()

In [19]:
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


## Interoperability with Numpy

In [20]:
import numpy as np
x = np.array([[1,2],[3,4]])

In [21]:
y = torch.from_numpy(x)

In [22]:
x

array([[1, 2],
       [3, 4]])

In [23]:
y

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

In [24]:
z = y.numpy()

In [25]:
z

array([[1, 2],
       [3, 4]])

## Linear Regression

### Custom Input Data

In [26]:
inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]],dtype = 'float32')

In [27]:
targets = np.array([[56,70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119]],dtype = 'float32')

In [28]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Linear Regression model from scratch

 X x W^T + b

In [31]:
w = torch.randn(2,3,requires_grad = True)
b = torch.randn(2, requires_grad = True)

In [32]:
print(w)
print(b)

tensor([[-2.9588,  1.1418,  1.3417],
        [-0.2942,  0.3257, -0.8685]], requires_grad=True)
tensor([1.1158, 0.6787], requires_grad=True)


##### @ represents Matrix Multiplication in Pytorch

In [33]:
def model(x):
    return x@ w.t() + b

In [35]:
preds = model(inputs)

In [36]:
print(preds)

tensor([[ -80.6829,  -36.3208],
        [ -81.7882,  -53.0153],
        [ -25.4811,  -31.6435],
        [-201.9400,  -47.4588],
        [   0.4890,  -49.1483]], grad_fn=<AddBackward0>)


In [37]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Loss Function

We are using MSE

In [38]:
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff * diff)/diff.numel()

.numel() returns the number of elements in a tensor

In [39]:
loss = mse(preds,targets)

In [40]:
loss

tensor(22425.4102, grad_fn=<DivBackward0>)

In [41]:
torch.sqrt(loss)

tensor(149.7512, grad_fn=<SqrtBackward0>)

### Compute Gradients w.r.t Loss

In [42]:
loss.backward()

In [43]:
print(w)

tensor([[-2.9588,  1.1418,  1.3417],
        [-0.2942,  0.3257, -0.8685]], requires_grad=True)


In [44]:
print(w.grad)

tensor([[-13455.3154, -12462.8125,  -8027.4536],
        [-11263.5664, -12502.6074,  -7774.6909]])


Reset the gradients to zero using .zero_() since Pytorch accumualtes gradients

In [45]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

Implementing all above step by step

In [46]:
preds = model(inputs)

In [47]:
loss = mse(preds,targets)
loss.backward()

##### Adjusting weights and resetting gradients

In [48]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [49]:
print(w)
print(b)

tensor([[-2.8242,  1.2664,  1.4219],
        [-0.1816,  0.4508, -0.7908]], requires_grad=True)
tensor([1.1174, 0.6801], requires_grad=True)


In [51]:
preds = model(inputs)
loss = mse(preds,targets)
loss

tensor(15645.5684, grad_fn=<DivBackward0>)

In [52]:
print(torch.sqrt(loss))

tensor(125.0822, grad_fn=<SqrtBackward0>)


### Train for Multiple epochs

In [53]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [54]:
preds = model(inputs)
loss = mse(preds,targets)
print(torch.sqrt(loss))

tensor(23.5075, grad_fn=<SqrtBackward0>)


### Linear Regression using Pytorch Built Ins

In [55]:
import torch.nn as nn

In [56]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [57]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Dataset and DataLoader

In [62]:
from torch.utils.data import TensorDataset,DataLoader

In [59]:
train_ds = TensorDataset(inputs,targets)

In [61]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [63]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle = True)

In [64]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5216,  0.4796, -0.3508],
        [-0.1460,  0.5094, -0.1556]], requires_grad=True)
Parameter containing:
tensor([0.4275, 0.0939], requires_grad=True)


In [65]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.5216,  0.4796, -0.3508],
         [-0.1460,  0.5094, -0.1556]], requires_grad=True),
 Parameter containing:
 tensor([0.4275, 0.0939], requires_grad=True)]

In [66]:
preds = model(inputs)
preds

tensor([[-20.6034,  16.8773],
        [-27.2878,  21.6799],
        [ -1.0355,  46.6285],
        [-45.1367,   1.3526],
        [-14.0795,  28.0328],
        [-21.6046,  16.2219],
        [-28.1182,  21.0150],
        [ -1.9079,  46.3270],
        [-44.1355,   2.0079],
        [-13.9086,  28.0232],
        [-21.4337,  16.2123],
        [-28.2891,  21.0246],
        [ -0.2051,  47.2934],
        [-45.3076,   1.3622],
        [-13.0782,  28.6882]], grad_fn=<AddmmBackward0>)

In [67]:
# Loss Function
import torch.nn.functional as F
loss_fn = F.mse_loss

In [68]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(7627.3862, grad_fn=<MseLossBackward0>)


In [69]:
print(torch.sqrt(loss))

tensor(87.3349, grad_fn=<SqrtBackward0>)


#### Optimizer

In [70]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

### Train the Model

In [71]:
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            # Generate the predictions
            pred = model(xb)
            # Calculate loss
            loss = loss_fn(pred,yb)
            # Compute gradients
            loss.backward()
            # Update parameters using gradients
            opt.step()
            # Reset gradients to zero
            opt.zero_grad()
        if (epoch + 1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [73]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 51.6128
Epoch [20/100], Loss: 62.0771
Epoch [30/100], Loss: 34.6821
Epoch [40/100], Loss: 19.6970
Epoch [50/100], Loss: 15.7266
Epoch [60/100], Loss: 37.0340
Epoch [70/100], Loss: 35.3535
Epoch [80/100], Loss: 37.1161
Epoch [90/100], Loss: 8.7070
Epoch [100/100], Loss: 14.2264


In [74]:
preds = model(inputs)
preds

tensor([[ 57.3523,  70.7925],
        [ 78.9810,  97.2293],
        [124.2273, 139.3930],
        [ 22.6036,  39.8387],
        [ 95.6143, 111.5678],
        [ 56.0447,  69.6532],
        [ 78.2865,  96.7217],
        [124.2169, 139.6663],
        [ 23.9112,  40.9780],
        [ 96.2274, 112.1995],
        [ 56.6578,  70.2850],
        [ 77.6734,  96.0900],
        [124.9218, 139.9006],
        [ 21.9905,  39.2070],
        [ 96.9219, 112.7071]], grad_fn=<AddmmBackward0>)